In [3]:
import sympy
import sequences
import igraph
from igraph import Graph, EdgeSeq
import plotly.graph_objects as go

In [52]:
g = sympy.Function('G')
c = sympy.Function('C')
q = sympy.Function('Q')
t = sympy.Function('T')
h = sympy.Function('H')
n = sympy.symbols('n')
f = sympy.Function('F')
gol = sequences.Recurrence(g(n-g(n-1)) + 1, g, n)
conn = sequences.Recurrence(c(n-c(n-1)) + c(n-1-c(n-2)), c, n)
hofs = sequences.Recurrence(q(n-q(n-1)) + q(n-q(n-2)), q, n)
tanny = sequences.Recurrence(t(n-1-t(n-1))+t(n-2-t(n-2)), t, n)
new = sequences.Recurrence(h(n-1-h(n-2))+h(n-3-h(n-4)), h, n)

neg = sequences.Recurrence(f(n+2-f(n-1))+f(n+1-f(n-2)),f,n)

In [5]:
sequences.seq_terms(sequences.Sequence([1,2,2,3,4,4,4]), new, 1, 101)

[1, 2, 2, 3, 4, 4, 4, 4, 5, 6, 7, 8, 8, 8, 8, 8, 8, 9, 10, 11, 12, 12, 12, 13, 14, 15, 16, 16, 16, 16, 16, 16, 16, 16, 17, 18, 19, 20, 20, 20, 21, 22, 23, 24, 24, 24, 24, 24, 25, 26, 27, 28, 28, 28, 29, 30, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 34, 35, 36, 36, 36, 37, 38, 39, 40, 40, 40, 40, 40, 41, 42, 43, 44, 44, 44, 45, 46, 47, 48, 48, 48, 48, 48, 48, 48, 49, 50, 51]

In [6]:
tsequences.seq_terms(sequences.Sequence([1,1]), gol, 1, 101)

In [53]:
sequences.seq_terms(sequences.Sequence([4,5,6,6,7]),neg,1,101)

[4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12, 12, 13, 14, 14, 15, 16, 16, 16, 17, 18, 18, 19, 20, 20, 20, 21, 22, 22, 23, 24, 24, 24, 24, 25, 26, 26, 27, 28, 28, 28, 29, 30, 30, 31, 32, 32, 32, 32, 33, 34, 34, 35, 36, 36, 36, 37, 38, 38, 39, 40, 40, 40, 40, 41, 42, 42, 43, 44, 44, 44, 45, 46, 46, 47, 48, 48, 48, 48, 48, 49, 50, 50, 51, 52, 52, 52, 53, 54, 54, 55, 56, 56, 56, 56, 57, 58, 58]

In [30]:
sequences.seq_terms(sequences.Sequence([10,10,11,12,12,12,13,14,14,15,16,17,18,18,19]),neg,1,101)

[10, 10, 11, 12, 12, 12, 13, 14, 14, 15, 16, 17, 18, 18, 19, 20, 20, 20, 21, 22, 22, 23, 24, 24, 24, 24, 25, 26, 26, 27, 28, 28, 28, 29, 30, 30, 31, 32, 32, 33, 34, 34, 35, 36, 36, 36, 37, 38, 38, 39, 40, 40, 40, 40, 41, 42, 42, 43, 44, 44, 44, 45, 46, 46, 47, 48, 48, 48, 48, 48, 49, 50, 50, 51, 52, 52, 52, 53, 54, 54, 55, 56, 56, 56, 56, 57, 58, 58, 59, 60, 60, 60, 61, 62, 62, 63, 64, 64, 64, 65]

In [66]:
# used to keep track of the value of the nodes during traversed and the tree's leaves 
global num
global leaves

class Node:
# creates a node with a specific value (stored as a list), children (stored as a list), and a height
# self.supernode keeps track of if a specific node is a supernode or not 
    def __init__(self,height = 0, supernode = True):
        self.value = []
        self.children = []
        self.height = height
        self.supernode = supernode
        
# creates a tree skeleton using the node it is called upon as the root 
# num_children determines how many children each node has 
    def make_skeleton(self, num_children):
        # as long as the height of the node is greater than 0, this will add n children to self.children 
        # if the node is a supernode, that node's left child is also a supernode 
        for i in range(0,num_children):
            if self.supernode==True and i == 0 and self.height > 0:
                self.children.append(Node(height = self.height - 1, supernode=True))
            elif self.height > 0:
                self.children.append(Node(height = self.height - 1, supernode = False))
        # calls the same function on the node's children, as long as their height is >0 
        for child in self.children:
            if child.height > 0:
                child.make_skeleton(num_children)
    
    # resets num, and then calls a function which labels the skeleton 
    # each of the parameters detemines the number of values in supernodes, non-supernodes, and leaves 
    def label_skeleton(self,supernode_values=1,nonsupernode_values=1,leave_values=1):
        global num 
        num = 1
        Node.call_label_skeleton(self,supernode_values,nonsupernode_values,leave_values)
        
    # traverses the tree and labels each node 
    def call_label_skeleton(self,supernode_values,nonsupernode_values,leave_values):
        global num
        # if the node is a supernode with height > 0, calls the function on the left child, labels itself, 
        # and then calls the function on its other children 
        if self.supernode==True and self.height >0:
            Node.call_label_skeleton(self.children[0],supernode_values,nonsupernode_values,leave_values)
            for i in range(0,supernode_values):
                self.value.append(num)
                num+= 1
            for child in self.children[1:]:
                Node.call_label_skeleton(child,supernode_values,nonsupernode_values,leave_values)
        # if the node has height > 0 is not a supernode, labels itself and the calls upon its children
        elif self.height > 0 and self.supernode == False:
            for i in range(0,nonsupernode_values):
                self.value.append(num)
                num+=1
            for child in self.children:
                Node.call_label_skeleton(child,supernode_values,nonsupernode_values,leave_values)
        else:
        # if height == 0, the node labels itself 
            for i in range(0, leave_values):
                self.value.append(num)
                num+=1
    
    # resets num, and then calls a function which labels the tree using negative supernodes 
    # num_negatives determines the number of labels in each supernode 
    def neg_labels(self,num_negatives):
        global num
        num = 1
        Node.call_neg_labels(self, num_negatives)
    
    # traverses the tree and labels each node. supernodes "undo" the previous n labels determined by num_negatives
    def call_neg_labels(self, num_negatives):
        global num
        #if a node is a supernode and height >0, calls the function on its left child
        #then undos the previous n labels (these become that nodes labels) and calls the function on other children
        if self.supernode==True and self.height>0:
            Node.call_neg_labels(self.children[0],num_negatives)
            num -=1
            self.value.append(num)
            for i in range(0,num_negatives-1):
                num -=1
                self.value.append(num)
            for child in self.children[1:]:
                Node.call_neg_labels(child,num_negatives)
        # the rest of the function proceeds the same as the label_skeleton function
        elif self.height > 0 and self.supernode == False:
            self.value.append(num)
            num+=1
            for child in self.children:
                Node.call_neg_labels(child,num_negatives)
        else:
                self.value.append(num)
                num+=1        
            
    # this function prints the labels of each node in the lable they would be traversed#
    # mostly used for debugging purposes #
    def print_tree(self):
        if self.height > 0 and self.supernode==True:
            self.children[0].print_tree()
            print(str(self.value),end='')
            for child in self.children[1:]:
                child.print_tree()
        elif self.height > 0 and self.supernode == False:
            print(str(self.value), end = '')
            for child in self.children:
                child.print_tree()
        else:
            print(str(self.value),end= '' )
        return ''
    
    # this is old and only works for positive supernode labels
    def old_find_sequence(self): 
        global leaves
        leaves = 0
        return Node.old_call_find_sequence(self,[])
        
    # this is old and only works for positive supernode labels    
    def old_call_find_sequence(self, list):
        global leaves
        if self.height > 0 and self.supernode==True:
            self.children[0].old_call_find_sequence(list)
            for i in self.value:
                list.append(leaves)
            for child in self.children[1:]:
                child.old_call_find_sequence(list)
        elif self.height > 0 and self.supernode == False:
            for i in self.value:
                list.append(leaves)
            for child in self.children:
                child.old_call_find_sequence(list)
        else:
            leaves += 1
            for i in self.value:
                list.append(leaves)
        return list
    
    # calls upon helper function to create a list of leaves
    # seq is the sequence represented by the tree, place_in_seq is the current position in the sequence
    # leaves_less_than is the number of leaves less than the current position in the sequence
    # new_list_of_leaves helps remove leaves that are less than the current position
    def find_sequence(self):
        list_of_leaves = Node.call_find_sequence(self,[])
        seq = []
        place_in_seq = 0
        leaves_less_than = 0
        new_list_of_leaves = []
    # continues looping through the list of leaves until it is empty
    # 
        while len(list_of_leaves) != 0:
            for item in list_of_leaves:
                if item <= place_in_seq + 1:
                    leaves_less_than += 1
                else:
                    new_list_of_leaves.append(item)
            seq.append(leaves_less_than)
            place_in_seq += 1 
            list_of_leaves=new_list_of_leaves
            new_list_of_leaves=[]
        return seq
                
            
        
    # goes through each node in the tree and creates a list of leaves    
    def call_find_sequence(self,list_of_leaves):
        if self.height>0: 
            for child in self.children:
                child.call_find_sequence(list_of_leaves)
        elif self.height == 0:
            list_of_leaves.append(self.value[0])
        return list_of_leaves
            
                        
## test ##
node1 = Node(height=5)
node1.make_skeleton(2)
node1.neg_labels(2)    
print(node1.find_sequence())

[4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12, 12, 13, 14, 14, 15, 16, 16, 16, 17, 18, 18, 19, 20, 20, 20, 21, 22, 22, 23, 24, 24, 24, 24, 25, 26, 26, 27, 28, 28, 28, 29, 30, 30, 31, 32]


In [68]:
def conn(n):
    next_vert  = 1
    root = 0
    tree = Graph()
    tree.add_vertex(0)
    for i in range(n):
        tree.add_vertex(next_vert)
        tree.add_edge(root,next_vert)
        next_root = next_vert
        next_vert += 1
        # traverse what we have to make copy #
        visited = {}
        parents = {}
        to_process = [root]
        while len(to_process) > 0:
            cur_vert = to_process[0]
            to_process = to_process[1:]
            tree.add_vertex(next_vert)
            visited[cur_vert] = next_vert
            if cur_vert == root:
                tree.add_edge(next_vert,next_root)
            else:
                tree.add_edge(next_vert,parents[cur_vert])
            for v in tree.neighbors(cur_vert):
                if v not in visited and v != next_root:
                    parents[v] = next_vert
                    to_process.append(v)
            next_vert +=1
        root = next_root
    return tree
    
        
        

In [69]:
print(conn(3))

IGRAPH UN-- 15 14 --
+ attr: name (v)
+ edges (vertex names):
0--1, 1--2, 1--3, 3--4, 4--5, 4--6, 3--7, 7--8, 8--9, 8--10, 9--11, 9--12,
10--13, 10--14


In [70]:
tree1 = conn(3)
tree1.get_edgelist()
#tree1_lay = tree1.layout_sugiyama(layers=[3,2,3,1,2,3,3,0,1,2,2,3,3,3,3])
#igraph.plot(tree1,layout=tree1_lay)

[(0, 1),
 (1, 2),
 (1, 3),
 (3, 4),
 (4, 5),
 (4, 6),
 (3, 7),
 (7, 8),
 (8, 9),
 (8, 10),
 (9, 11),
 (9, 12),
 (10, 13),
 (10, 14)]